In [1]:
#Метрика и баланс по классам нужно построить
#Разобраться с knn_acc_score


# Импорт базовых библиотек
import numpy as np
import pandas as pd

# Визуализация

import seaborn as sns
import matplotlib.pyplot as plt

# Utils
from sklearn.model_selection import train_test_split, cross_val_score

# Модели (классификация)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Модели (регрессия)
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# Метрики
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score  # для классификации
from sklearn.metrics import mean_absolute_error, mean_squared_error  # для регрессии

# Misc
from sklearn.manifold import TSNE

In [2]:
#Загрузка базы данных

dfm = pd.read_csv('./SpotifyFeatures.csv')

#dfm.head(13)
dfm.shape

(232725, 18)

In [21]:
#Создание и сохранение базы данных с категориями песен

#for n in dfm.index:
    #if 0 <= dfm.iloc[n].popularity <= 20 :
        #dfm.loc[n,'Category'] = 'Popular'
    #elif 20 < dfm.iloc[n].popularity <= 50:
        #dfm.loc[n,'Category'] = 'Usual'
    #elif 51 < dfm.iloc[n].popularity:
        #dfm.loc[n,'Category'] = 'Unpopular'
#dfm.to_csv('./SpotifyFeatures_new.csv')

In [3]:
dfm_P = pd.read_csv('./SpotifyFeatures_new.csv')


#Зададим количество строчек для модели

n = 200

In [4]:
df=dfm_P.head(n)
df

,Unnamed: 0,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,Category
0,0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.6110,0.389,99373,0.910,0.000000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814,Popular
1,1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.2460,0.590,137373,0.737,0.000000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816,Popular
2,2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.9520,0.663,170267,0.131,0.000000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368,Popular
3,3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.7030,0.240,152427,0.326,0.000000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227,Popular
4,4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.9500,0.331,82625,0.225,0.123000,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390,Popular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,R&B,Kelly Clarkson,Love So Soft,16QN8cBubEq706vNtPpOME,67,0.0053,0.808,172427,0.581,0.000012,B,0.0828,-5.430,Major,0.0700,130.012,4/4,0.758,Unpopular
196,196,R&B,PARTYNEXTDOOR,Not Nice,4STaHtfguDMkWbPEo6Fwcx,69,0.0253,0.598,202661,0.496,0.001320,C,0.1200,-9.309,Major,0.0804,128.060,4/4,0.725,Unpopular
197,197,R&B,Tory Lanez,Pa Mi (with Ozuna),0CYLDwvvpJQySLA5CvqvSl,71,0.0983,0.596,216689,0.557,0.000000,C#,0.0565,-7.522,Minor,0.1340,133.247,5/4,0.661,Unpopular
198,198,R&B,The Weeknd,A Lonely Night,0ifooM33X1iBxVYyOkSbXW,68,0.0431,0.853,220173,0.663,0.000009,G,0.0891,-6.955,Minor,0.0755,113.999,4/4,0.864,Unpopular


In [5]:
from scipy import stats
print(stats.normaltest(df['tempo']))
print(stats.normaltest(df['danceability']))
print(stats.normaltest(df['energy']))

NormaltestResult(statistic=5.1007339723470775, pvalue=0.07805301636665526)
NormaltestResult(statistic=9.41196596669845, pvalue=0.009041022673750562)
NormaltestResult(statistic=8.068195179471529, pvalue=0.017701647055726438)


In [6]:
X = pd.DataFrame([df['tempo']])
X = X.T
X ['danceability'] = df['danceability']
X ['energy'] = df['energy']
X ['target'] = np.zeros(n)
y = df['Category']
#y = y.T

In [7]:
print(f'Type X: {type(X)}, X shape: {X.shape}')
print(f'Type y: {type(y)}, y shape: {y.shape}')

Type X: <class 'pandas.core.frame.DataFrame'>, X shape: (200, 4)
Type y: <class 'pandas.core.series.Series'>, y shape: (200,)


In [8]:
from sklearn.model_selection import cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, stratify=y, test_size=0.2)

# Инициализируем алгоритмы с параметрами по-умолчанию
knn_clf = KNeighborsClassifier()
svm_clf = SVC()
forest_clf = RandomForestClassifier()

# Посчитаем метрику качества используя кросс-валидацию
knn_acc_score = cross_val_score(knn_clf, X, y, cv=7).mean()
svm_acc_score = cross_val_score(svm_clf, X, y, cv=7).mean()
forest_acc_score = cross_val_score(forest_clf, X, y, cv=7).mean()

# Выведем алгоритмы по убыванию качества их работы
leaderboard = sorted([
    ('KNN', knn_acc_score),
    ('SVC', svm_acc_score),
    ('Forest', forest_acc_score)
], key=lambda pair: pair[1], reverse=True)

print('Leaderboard:')
for idx, (leader_name, leader_score) in enumerate(leaderboard):
    print(f'{idx + 1}. {leader_name}: {leader_score}')

Leaderboard:
1. SVC: 0.6750527797325827
2. Forest: 0.6600985221674877
3. KNN: 0.5849753694581281
